<a href="https://colab.research.google.com/github/NguyenThuan215/ML-echo-cardiography/blob/main/echocardiography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thư viện

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

from collections import namedtuple
from sklearn.metrics import classification_report

import torch
from torch import nn
from torch.utils.data import DataLoader

# Kết nối với data

In [2]:
!git clone https://github.com/NguyenThuan215/ML-echo-cardiography
traindir = "/content/ML-echo-cardiography/DATA_CHAMBER_2021/train"
testdir = "/content/ML-echo-cardiography/DATA_CHAMBER_2021/test"

Cloning into 'ML-echo-cardiography'...
remote: Enumerating objects: 8357, done.
remote: Total 8357 (delta 0), reused 0 (delta 0), pack-reused 8357
Receiving objects: 100% (8357/8357), 488.05 MiB | 30.69 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Checking out files: 100% (8328/8328), done.


# Chuẩn bị dữ liệu:
1. Các lớp: {2C, 3C, 4C}
2. Đọc dữ liệu trong file 'traindir' và 'testdir'
3. Đưa dữ liệu vào các batch để xử lý song song


In [9]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

#Generate different transformations 
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
hor_flip = transforms.RandomHorizontalFlip(p=0.3)
ver_flip = transforms.RandomVerticalFlip(p=0.3)
rotation_15 = transforms.RandomRotation(degrees=15)
rotation_10 = transforms.RandomRotation(degrees=10)
crop = transforms.RandomCrop(224, padding=4)
equalize = transforms.RandomEqualize(p=1)
blur = transforms.GaussianBlur(kernel_size=3)
resize_224 = transforms.Resize((224,224))
resize_64 = transforms.Resize((64,64))
resize_32 = transforms.Resize((32,32))

#transform for train set
transform_train_normal = transforms.Compose([resize_224, to_tensor])
transform_train_64 = transforms.Compose([resize_64, to_tensor])
transform_train_32 = transforms.Compose([resize_32, to_tensor])
transform_train_aug = transforms.Compose([resize_224, hor_flip, ver_flip, rotation_10, to_tensor])
transform_train_preprocess =  transforms.Compose([resize_224, equalize, blur, to_tensor])

#transform for test set
transform_test_normal = transforms.Compose([resize_224, to_tensor])
transform_test_64 = transforms.Compose([resize_64, to_tensor])
transform_test_32 = transforms.Compose([resize_32, to_tensor])
transform_test_preprocess =  transforms.Compose([resize_224, equalize, blur, to_tensor])

def creat_transforms(img_size=224, augmentation="None"):
  # img_size = 224,64,32
  # augmentation = "None", "preprocess", "augmentation"
  if augmentation == "None":
    if (img_size == 224): 
      transforms_train = transform_train_normal
      transforms_test = transform_test_normal
    elif (img_size == 64): 
      transforms_train = transform_train_64
      transforms_test = transform_test_64
    elif (img_size == 32): 
      transforms_train = transform_train_32
      transforms_test = transform_test_32
    else:
      pass
  elif augmentation == "preprocess":
    transforms_train = transform_train_preprocess
    transforms_test = transform_test_preprocess
  elif augmentation == "augmentation":
    transforms_train = transform_train_aug
    transforms_test = transform_test_normal
  else:
    pass
  print(transforms_train, transforms_test)
  return transforms_train, transforms_test

  
def prepare_data(transform_train=transform_train_normal, transform_test=transform_test_normal):
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch = 32
  worker = 4
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch, shuffle=True, num_workers=worker)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch, shuffle=False, num_workers=worker)
  return TrainTest(train=trainloader, test=testloader)

# Train/Test mô hình:
1. Train: với mỗi batch trong tập train:
  - Cho ảnh đi qua model
  - Tính lỗi bằng hàm lỗi "loss_func"
  - Cập nhật tham số
  - Báo cáo sau "reporting_steps" bước
2. Test:
  - Đặt model ở chế độ đánh giá (evaluate)
  - Tính toán đầu ra cho từng ảnh
  - trả về nhãn dự đoán/nhãn thực

In [10]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  final_loss = 0.0
  reporting_steps = 40
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    final_loss = loss.item()
    running_loss += final_loss
    if step % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return final_loss

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# Tạo và thực nghiệm mô hình:
1. Tham số truyền vào main:
  - "model_in": string thể hiện tên mô hình muốn thực nghiệm.
2. Sửa đổi đầu ra của lớp Linear cuối cùng thành "3" để phù hợp với yêu cầu bài toán
3. Sử dụng hàm lỗi CrossEntropyLoss, hàm tối ưu SGD (Stochastic Gradient Descent)


In [11]:
def creat_model(model_in=None, class_out=1000, device="cpu"):
  if model_in == 'vgg16':  
    model = torchvision.models.vgg16(pretrained=False)
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=class_out, bias=True)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50(pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=class_out, bias=True) 
  elif model_in == 'resnet18':
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=class_out, bias=True) 
  elif model_in == 'densenet121': 
    model = torchvision.models.densenet121(pretrained=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=class_out, bias=True)
  else:
    pass
    
  model.to(device=device)
  return model

In [50]:
def main(model_in, img_size, augmentation):
  PATH = "./" + model_in + '.pth'
  classes = get_classes()
  class_out = len(classes)

  transforms_train, transforms_test = creat_transforms(img_size,augmentation)
  datasets = prepare_data(transforms_train, transforms_test)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  print("image size: ", datasets.train[0][0].shape)
  print("device:", device)
  print("model:", model_in)

  model = creat_model(model_in=model_in, class_out=class_out, device=device) 
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  accuracies = []
  losses = []
  for epoch in range(20):
    print("---------------------------------------------------------------")
    loss = train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)

    print(classification_report(ytrue_test, ypred_test, target_names=classes))
    torch.save(model.state_dict(), PATH)

    # calculate report
    ypred_test = np.array(ypred_test)
    ytrue_test = np.array(ytrue_test)
    accuracy = int((ytrue_test==ypred_test).sum() / len(ytrue_test) *100)
    accuracies.append(accuracy)
    losses.append(round(loss, 4))

    
  print("model:", model_in, ", size:", img_size, ", augmentation:", augmentation)
  print("accr: ", accuracies)
  print("loss: ", losses)
  return model

In [52]:
models = ['vgg16', 'resnet18', 'resnet50', 'densenet121']
img_sizes = [224, 64, 32]
augmentation = ['None', "preprocess", "augmentation"]

for i in models:
  model = main(model_in=i, img_size=224,augmentation="None")

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
)
image size:  torch.Size([3, 224, 224])
device: cuda
model: vgg16


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


---------------------------------------------------------------
Epoch 0 step 39 ave_loss 1.0451
Epoch 0 step 79 ave_loss 1.0439
Epoch 0 step 119 ave_loss 1.0343
Epoch 0 step 159 ave_loss 0.6680
Epoch 0 step 199 ave_loss 0.4956
              precision    recall  f1-score   support

          2C       0.63      0.41      0.50       409
          3C       0.44      0.97      0.60       367
          4C       1.00      0.63      0.77       831

    accuracy                           0.65      1607
   macro avg       0.69      0.67      0.63      1607
weighted avg       0.78      0.65      0.67      1607

---------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 39 ave_loss 0.2354
Epoch 1 step 79 ave_loss 0.2529
Epoch 1 step 119 ave_loss 0.3077
Epoch 1 step 159 ave_loss 0.1101
Epoch 1 step 199 ave_loss 0.0682
              precision    recall  f1-score   support

          2C       0.94      0.78      0.85       409
          3C       0.62      0.99      0.76       367
          4C       1.00      0.81      0.90       831

    accuracy                           0.85      1607
   macro avg       0.85      0.86      0.84      1607
weighted avg       0.90      0.85      0.85      1607

---------------------------------------------------------------


...

Epoch 17 step 39 ave_loss 0.0004
Epoch 17 step 79 ave_loss 0.0004
Epoch 17 step 119 ave_loss 0.0004
Epoch 17 step 159 ave_loss 0.0003
Epoch 17 step 199 ave_loss 0.0005
              precision    recall  f1-score   support

          2C       0.90      0.80      0.85       409
          3C       0.68      0.96      0.80       367
          4C       1.00      0.88      0.93       831

    accuracy                           0.88      1607
   macro avg       0.86      0.88      0.86      1607
weighted avg       0.90      0.88      0.88      1607

---------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 18 step 39 ave_loss 0.0002
Epoch 18 step 79 ave_loss 0.0004
Epoch 18 step 119 ave_loss 0.0003
Epoch 18 step 159 ave_loss 0.0002
Epoch 18 step 199 ave_loss 0.0002
              precision    recall  f1-score   support

          2C       0.92      0.80      0.85       409
          3C       0.66      0.95      0.78       367
          4C       1.00      0.87      0.93       831

    accuracy                           0.87      1607
   macro avg       0.86      0.87      0.86      1607
weighted avg       0.90      0.87      0.88      1607

---------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 19 step 39 ave_loss 0.0003
Epoch 19 step 79 ave_loss 0.0004
Epoch 19 step 119 ave_loss 0.0009
Epoch 19 step 159 ave_loss 0.0005
Epoch 19 step 199 ave_loss 0.0004
              precision    recall  f1-score   support

          2C       0.84      0.81      0.82       409
          3C       0.66      0.90      0.76       367
          4C       1.00      0.85      0.92       831

    accuracy                           0.85      1607
   macro avg       0.83      0.85      0.84      1607
weighted avg       0.88      0.85      0.86      1607

model: densenet121 , size: 224 , augmentation: None
accr:  [91, 75, 81, 86, 86, 87, 89, 88, 88, 87, 88, 88, 88, 88, 88, 86, 86, 87, 87, 85]
loss:  [0.1349, 0.0016, 0.0004, 0.0002, 0.0002, 0.0001, 0.0007, 0.0013, 0.0001, 0.0001, 0.0001, 0.0, 0.0002, 0.0002, 0.0002, 0.0001, 0.0001, 0.003, 0.0, 0.0011]


## Test mô hình đã huấn luyện trên video
  - Lấy ra toàn bộ các frame của 1 video
  - cho các frame đã có qua mô hình đã huấn huyện -> kết quả được 1 mảng thể hiện các frame được dự đoán ở lớp nào
  - Đếm giá trị các phân lớp, lấy giá trị phân lớp có nhiều frame được phân vào nhất làm giá trị phân lớp của video

Ví dụ: Video 123 có 21 frame:
  - 18 frame được dự đoán là lớp 2C (86%)
  - 2 frame được dự đoán là lớp 3C (10%)
  - 1 frame được dự đoán là lớp 4C (4%)

-> kết quả phân lớp của video 123 là lớp 2C



In [55]:
def video_report(model=None, testsets=None, device="cuda"):
  if testsets == None:
    testsets = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_64)

  video = namedtuple('video', ['id', "label_true", 'label_pred'])
  path = [] # [158, 158, 158, 165, 165, 135, 135 ...]
  for index, image in enumerate(testsets.imgs):
    path.append(image[0].split("/")[-1].split("_")[0])

  vid_list = [] # [158, 165, 135 ...]
  for frame in path:
    if (frame in vid_list) == False:
      vid_list.append(frame)
    else:
      pass

  ytrue = []
  ypred = []
  model.to(device)
  with torch.no_grad():
    model.eval()
    for image, label in testsets:
      image = image.unsqueeze(0).to(device)
      output = model(image)
      _, predicted = torch.max(output, dim=1)
      ytrue.append(label)
      ypred += list(predicted.cpu().numpy())

  outputs_vid = []
  ytrue_video = []
  ypred_video = []
  for vid_id in vid_list:
    vid_true = []
    vid_pred = []
    for index, img in enumerate(path):
      if img == vid_id:
        vid_true.append(ytrue[index])
        vid_pred.append(ypred[index])

    rate_video_true = [0, 0, 0] # vector count for voting
    rate_video_pred = [0, 0, 0]
    for label in range(3):
      rate_video_true[label] = list(vid_true).count(label)
      rate_video_pred[label] = list(vid_pred).count(label)
    
    label_video_true = np.argmax(rate_video_true)
    label_video_pred = np.argmax(rate_video_pred)

    print("id:", vid_id, ", true:", label_video_true, ", pred:",label_video_pred) # label pred
    ytrue_video.append(label_video_true)
    ypred_video.append(label_video_pred)
    outputs_vid.append(video(id=vid_id, label_true=label_video_true, label_pred=label_video_pred))
  print(classification_report(ytrue_video, ypred_video, target_names=get_classes()))
  return outputs_vid


In [56]:
testsets = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_normal)
video_report(model=model, testsets=testsets,device ="cuda")
# Bên dưới là kết quả thực hiện phân lớp bằng video của mô hình DENSENET121 với ảnh resize 224x224, không nhiễu, không tiền xử lý

id: 158 , true: 0 , pred: 0
id: 165 , true: 0 , pred: 0
id: 168 , true: 0 , pred: 0
id: 169 , true: 0 , pred: 0
id: 171 , true: 0 , pred: 0
id: 176 , true: 0 , pred: 0
id: 177 , true: 0 , pred: 0
id: 178 , true: 0 , pred: 0
id: 181 , true: 0 , pred: 1
id: 183 , true: 0 , pred: 0
id: 191 , true: 0 , pred: 0
id: 192 , true: 0 , pred: 0
id: 157 , true: 1 , pred: 1
id: 159 , true: 1 , pred: 1
id: 161 , true: 1 , pred: 1
id: 162 , true: 1 , pred: 1
id: 166 , true: 1 , pred: 1
id: 174 , true: 1 , pred: 0
id: 175 , true: 1 , pred: 1
id: 179 , true: 1 , pred: 1
id: 185 , true: 1 , pred: 1
id: 186 , true: 1 , pred: 1
id: 189 , true: 1 , pred: 1
id: 190 , true: 1 , pred: 1
id: 194 , true: 1 , pred: 1
id: 160 , true: 2 , pred: 1
id: 163 , true: 2 , pred: 2
id: 164 , true: 2 , pred: 2
id: 167 , true: 2 , pred: 2
id: 170 , true: 2 , pred: 2
id: 172 , true: 2 , pred: 2
id: 173 , true: 2 , pred: 2
id: 180 , true: 2 , pred: 2
id: 182 , true: 2 , pred: 2
id: 184 , true: 2 , pred: 2
id: 187 , true: 2 , 

[video(id='158', label_true=0, label_pred=0),
 video(id='165', label_true=0, label_pred=0),
 video(id='168', label_true=0, label_pred=0),
 video(id='169', label_true=0, label_pred=0),
 video(id='171', label_true=0, label_pred=0),
 video(id='176', label_true=0, label_pred=0),
 video(id='177', label_true=0, label_pred=0),
 video(id='178', label_true=0, label_pred=0),
 video(id='181', label_true=0, label_pred=1),
 video(id='183', label_true=0, label_pred=0),
 video(id='191', label_true=0, label_pred=0),
 video(id='192', label_true=0, label_pred=0),
 video(id='157', label_true=1, label_pred=1),
 video(id='159', label_true=1, label_pred=1),
 video(id='161', label_true=1, label_pred=1),
 video(id='162', label_true=1, label_pred=1),
 video(id='166', label_true=1, label_pred=1),
 video(id='174', label_true=1, label_pred=0),
 video(id='175', label_true=1, label_pred=1),
 video(id='179', label_true=1, label_pred=1),
 video(id='185', label_true=1, label_pred=1),
 video(id='186', label_true=1, lab